# Generator

In [1]:
%matplotlib inline
import numpy as np
import os
from timeit import default_timer as timer
import image_tools as imt
from augmentator import Augmentator
from generator import Generator

In [2]:
image_shape = [128,128,1]
images_dir  = "/netforge/datasets/private/roman/coins/2017-11-29/images/"
dataset_dir = "/netforge/datasets/private/roman/coins/2017-11-29/datasets/"

## Generate

In [ ]:
%%time 
gen = Generator()
try:
    start = timer()
    gen.generate(
        images_path = images_dir, 
        image_shape = image_shape, 
        augmentator = Augmentator(), 
        image_num   = 500,
        aug_num     = 300, 
    )
    t = timer() - start
    augmented = gen.augmented
    n = augmented.shape[0]*augmented.shape[1]
    print("Augmented  : %d images" % n)
    print("Performance: %.1f img/sec" % (n/t))
    print("Augmented  :", augmented.dtype, augmented.shape, augmented.nbytes/1024/1024, "MB")
except KeyboardInterrupt:
    print("Interrupted")
print("-"*50)

  2%|▏         | 6/300 [00:15<13:01,  2.66s/it]

## Save

In [ ]:
%%time
dataset_path = gen.save(dataset_dir)
print("Dataset:", dataset_path)

## Load

In [ ]:
%%time
try:
    del gen
except:
    pass
data = np.load(dataset_path)
print("Data:", data.dtype, data.shape, data.nbytes/1024/1024, "MB")
print("-"*50)

## Show

In [ ]:
images = data[7,:24]
images = imt.uint8_to_float32(images)
imt.plot(images, 6)